<h2> Featurizing text data with tfidf weighted word-vectors </h2>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings("ignore")
import sys
import os
import pandas as pd
import numpy as np
from tqdm import tqdm


import spacy

In [ ]:
# !python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 865.1 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:

df = pd.read_csv("/content/drive/MyDrive/Project_gova/train.csv")

# df = df[[	'qid1','qid2','question1',"question2"]]

df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))

In [4]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,325999,66488,63609,What are the safety precautions on handling sh...,What are the safety precautions on handling sh...,1
1,85974,145050,145051,Which type of question asked by Amazon intervi...,Should I co-author a book with my marketing pr...,0
2,277216,169290,6439,Where can I download free music?,How do I download free music?,1
3,88878,149407,149408,Why do some people hate Starbucks?,Why do people hate Starbucks?,1
4,166859,258801,258802,What is the most real life like movie about pi...,What are the most epic T-shirt quotes you have...,0


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(df['question1']) + list(df['question2'])

tfidf = TfidfVectorizer(lowercase=False, )
tfidf.fit_transform(questions)

print(tfidf.get_feature_names_out(),len(tfidf.get_feature_names_out()))
# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names_out(), tfidf.idf_))

['00' '000' '0000000000' ... '이정현' '친구해도' '하지만'] 57352


- After we find TF-IDF scores, we convert each question to a weighted average of word2vec vectors by these scores.
- here we use a pre-trained GLOVE model
- It is trained on Wikipedia and therefore, it is stronger in terms of word semantics.

In [6]:
len(word2tfidf)

57352

In [7]:
df_q = pd.read_csv('/content/drive/MyDrive/Project_gova/external_test.csv')

In [8]:
# en_vectors_web_sm, which includes over 1 million unique vectors.
nlp = spacy.load('en_core_web_sm')

vecs1 = []

# tqdm is used to print the progress bar
for qu1 in tqdm(list(df_q['question1'])):
    doc1 = nlp(qu1)
    # 384 is the number of dimensions of vectors
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])
    print(len(mean_vec1))
    for word1 in doc1:
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
    print(vecs1)

df_q['q1_feats_m'] = list(vecs1)

print(len(vecs1))

100%|██████████| 1/1 [00:00<00:00, 44.79it/s]

5
[array([-15.93712276, -16.27733123,   2.80084127,  10.60668185,
       -13.42874305, -16.31017792,  13.20772713, -10.33092007,
        19.58460045,   1.99085867,   6.76156341, -36.77107239,
         5.10801816,  19.08068565, -18.67253828, -10.26957703,
        -7.10453439, -26.36012828,  15.47488952,   7.99279186,
       -30.5280056 ,  26.8225162 , -58.40699637, -20.63863873,
         2.36242551,  -7.08628368,   8.68852352,  13.8439377 ,
        -0.78061819,   6.96229899,  -6.41801345,  -1.67670119,
         6.36785471,  16.68579912,  12.40957332, -25.22678852,
        23.07411087,   7.72939825,  15.98976111, -41.93114007,
        -8.15033996,  -4.71168327, -29.51468301,  -3.71165597,
       -10.18801719, -10.16636682, -10.1906451 , -14.07806516,
        -4.65399742,   2.16562717,  -5.82571876,  16.45090459,
        -1.01817453,  -4.90391961, -10.51462317, -22.65229893,
        19.25020909,  17.5058299 ,  11.57338202,  -4.48656833,
        -6.14855123, -19.54410471,  -2.67911726,  -4

In [9]:
vecs2 = []
for qu2 in tqdm(list(df_q['question2'])):
    doc2 = nlp(qu2)
    print(len(doc2[0].vector))
    mean_vec2 = np.zeros([len(doc2), len(doc2[0].vector)])
    for word2 in doc2:
        # word2vec
        vec2 = word2.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word2)]
        except:
            #print word
            idf = 0
        # compute final vec
        mean_vec2 += vec2 * idf
    mean_vec2 = mean_vec2.mean(axis=0)
    vecs2.append(mean_vec2)
df_q['q2_feats_m'] = list(vecs2)

100%|██████████| 1/1 [00:00<00:00, 84.59it/s]

96


In [10]:
df_q.q1_feats_m[0].shape

(96,)

In [11]:
#prepro_features_train.csv (Simple Preprocessing Feartures)
#nlp_features_train.csv (NLP Features)
if os.path.isfile('/content/drive/MyDrive/Project_gova/external_nlp_feature_test.csv'):
    dfnlp = pd.read_csv("/content/drive/MyDrive/Project_gova/external_nlp_feature_test.csv",encoding='latin-1')
else:
    print("download nlp_features_test.csv from drive or run previous notebook")

if os.path.isfile('/content/drive/MyDrive/Project_gova/external_test.csv'):
    dfppro = pd.read_csv("/content/drive/MyDrive/Project_gova/external_test.csv",encoding='latin-1')
else:
    print("download df_fe_without_preprocessing_test.csv from drive or run previous notebook")

In [12]:
df1 = dfnlp.drop(['qid1','qid2','question1','question2'],axis=1)
df2 = dfppro.drop(['qid1','qid2','question1','question2'],axis=1)
df3 = df_q.drop(['qid1','qid2','question1','question2'],axis=1)
df3_q1 = pd.DataFrame(df3.q1_feats_m.values.tolist(), index= df3.index)
df3_q2 = pd.DataFrame(df3.q2_feats_m.values.tolist(), index= df3.index)

In [13]:
df1['id'] = 1
df2['id'] = 1

In [14]:
# dataframe of nlp features
df1.head()

,id,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,...,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
0,1,1,1,30,38,5,5,3.0,10.0,0.3,...,0.599988,1,0,0,5.0,85,71,76,86,0.741935


In [15]:
# data before preprocessing
df2.head()

,id,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2
0,1,1,1,30,38,5,5,3.0,10.0,0.3,2,0


In [16]:
# Questions 1 tfidf weighted word2vec
df3_q1.head()

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,-15.937123,-16.277331,2.800841,10.606682,-13.428743,-16.310178,13.207727,-10.33092,19.5846,1.990859,...,1.820027,24.923758,-6.454738,62.640062,21.214221,-17.334323,51.486448,15.012107,-29.720879,6.444104


In [17]:
# Questions 2 tfidf weighted word2vec
df3_q2.head()

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,6.357256,-35.669159,10.787417,5.660337,-10.783251,-9.755858,-6.181419,-8.921379,31.228711,7.983891,...,-1.225697,9.330784,1.954323,78.951339,24.946464,-21.663014,49.338879,29.904553,-31.689559,6.202816


In [18]:
print("Number of features in nlp dataframe :", df1.shape[1])
print("Number of features in preprocessed dataframe :", df2.shape[1])
print("Number of features in question1 w2v  dataframe :", df3_q1.shape[1])
print("Number of features in question2 w2v  dataframe :", df3_q2.shape[1])
print("Number of features in final dataframe  :", df1.shape[1]+df2.shape[1]+df3_q1.shape[1]+df3_q2.shape[1])

Number of features in nlp dataframe : 27
Number of features in preprocessed dataframe : 12
Number of features in question1 w2v  dataframe : 96
Number of features in question2 w2v  dataframe : 96
Number of features in final dataframe  : 231


In [19]:
# storing the final features to csv file
if not os.path.isfile('final_features.csv'):
    df3_q1['id']=df1['id']
    df3_q2['id']=df1['id']
    df1  = df1.merge(df2, on='id',how='left')
    df2  = df3_q1.merge(df3_q2, on='id',how='left')
    result  = df1.merge(df2, on='id',how='left')
    result.to_csv('/content/drive/MyDrive/Project_gova/external_final_features_test.csv')